# Experiment

In [ ]:
"""
import psutil
print(f'Number of physical cores: {psutil.cpu_count(logical=False)}')
"""
import os
def get_slurm_cpus():
    """
    Get the number of CPUs allocated by SLURM.
    
    Tries 'SLURM_CPUS_PER_TASK' first, then 'SLURM_CPUS_ON_NODE'.
    Returns None if neither is set.
    """
    # This is the most common variable for job steps
    cpus_per_task = os.getenv('SLURM_CPUS_PER_TASK')
    if cpus_per_task is not None:
        try:
            return int(cpus_per_task)
        except ValueError:
            print(f"Warning: Could not parse SLURM_CPUS_PER_TASK ('{cpus_per_task}')")
    
    # This is a fallback for the total CPUs on the node for the job
    cpus_on_node = os.getenv('SLURM_CPUS_ON_NODE')
    if cpus_on_node is not None:
        try:
            return int(cpus_on_node)
        except ValueError:
            print(f"Warning: Could not parse SLURM_CPUS_ON_NODE ('{cpus_on_node}')")
    
    print("Warning: SLURM CPU environment variables not set. ")
    print("Are you running this script within a SLURM job (sbatch/srun)?")
    return None

# --- Example Usage ---
allocated_cpus = get_slurm_cpus()

if allocated_cpus is not None:
    print(f"SLURM allocated {allocated_cpus} CPUs to this job/task.")
    # You can now use this number to configure libraries
    # e.g., os.environ['OMP_NUM_THREADS'] = str(allocated_cpus)
else:
    print("Could not determine SLURM CPU allocation.")

In [ ]:
%load_ext autoreload
%autoreload 2

Beware, this will result in, where we always save the `pipeline`, to be able to do restarts.
<pre>
    0:01 <span style="color:#42A5F5">  71K</span> S37_detector_data_slices.png
    0:01 <span style="color:#42A5F5"> 110K</span> S37_detector_data_sum.png
    0:01 <span style="color:#42A5F5"> 144K</span> S37_detector_data_orthogonalisation.png
    0:01 <span style="color:#FF7043"> 1.6G</span> S37_preprocessed_data.cxi
    0:01 <span style="color:#42A5F5"> 3.3K</span> preprocess_output.log
    0:12 <span style="color:#42A5F5">  53K</span> phase_retrieval_output.log
    0:15 <span style="color:#FF7043"> 9.6G</span> pynx_phasing/
    0:37 <span style="color:#B0B0B0">    0</span> mode_decomposition_output.log
    0:38 <span style="color:#FFA726"> 736M</span> S37_pynx_reconstruction_mode.cxi
    0:38 <span style="color:#B0B0B0">    0</span> postprocess_output.log
    0:39 <span style="color:#42A5F5">  78K</span> S37_amplitude_distribution.png
    0:39 <span style="color:#42A5F5"> 156K</span> S37_summary_plot.png
    0:39 <span style="color:#42A5F5"> 102K</span> S37_strain_methods.png
    0:39 <span style="color:#42A5F5">  88K</span> S37_shear_displacement.png
    0:39 <span style="color:#42A5F5"> 132K</span> S37_strain_statistics.png
    0:39 <span style="color:#42A5F5"> 203K</span> S37_3d_strain_views.png
    0:39 <span style="color:#42A5F5"> 158K</span> S37_final_object_fft.png
    0:39 <span style="color:#FFA726"> 869M</span> S37_postprocessed_data.cxi
    0:39 <span style="color:#66BB6A">  74M</span> S37_structural_properties.npz
    0:39 <span style="color:#66BB6A">  90M</span> S37_structural_properties.vti
    0:39 <span style="color:#42A5F5"> 2.3K</span> S37_parameters.yml
    0:39 <span style="color:#D32F2F">  17G</span> <b>pipeline.pkl</b>
</pre>

Thus, do not hoard computations for this part, as it is mostly a useless proof of concept for splitting random codes.

In [ ]:
import facets_drivers
import os
user = os.environ.get('USER')

local_run = False
notebook_data = {
    "num_cores": 48,
    "local_run": local_run,
    "experiment_directory": f"/data/id01/inhouse/{user}/scratch/ewoks_01_experiment/",  # last '/' important for cdiutil’s postprocess
    "experimental_structure": f"/data/id01/inhouse/{user}/scratch/ewoks_01_experiment/structural_properties.vti",
    "theory_directory": f"/data/id01/inhouse/{user}/scratch/ewoks_02_theory/",
    "simulation_directory": f"/data/id01/inhouse/{user}/scratch/ewoks_03_simulation/",
    "facets_directory": f"/data/id01/inhouse/{user}/scratch/ewoks_04_facets/",
    #"scale_factors": (4.0, 4.0, 4.0),  # 1.0: 10x smaller for simulation (angstrom instead of nanometers).
    "scale_factors": (1.0, 1.0, 1.0),
}
if local_run:
    notebook_data["num_cores"] = allocated_cpus

In [ ]:
import os
"""
simulation_parameters = {
    # metadata
    "beamline_setup": "p10", #id01spec
    "experiment_data_dir_path": (
        "/data/projects/carine/Experiments/P10_June24/raw/"
    ),
    "sample_name": "Pd_YSZ",  
    "scan": 37,
    
    # preprocessing parameters
    "preprocess_shape": (600, 400, 400),  # define cropped window size
    "voxel_reference_methods": [(300, 1326, 1489)],  # centring method sequence
    "background_level": 2,  # background intensity level to remove
    "hot_pixel_filter": True,  # remove isolated hot pixels
    "light_loading": True,
    "det_calib_params": {
        'distance': 1.82169,        # Sample-detector distance in meters
        'cch1': 1055.74,            # Direct beam position on detector (vert. pixel)
        'cch2': 1332.94,            # Direct beam position on detector (hor. pixel)
        'pwidth1': 7.5e-05,
        'pwidth2': 7.5e-05,  
    }
}
"""
"""
simulation_parameters = {
    # metadata
    "beamline_setup": "id01spec",
    # was 183
    "scan": 181,
    "sample_name": "Pt",

    # Paths for the 'id01spec' setup
    "experiment_file_path": (
        "/data/projects/carine/Experiments/ihhc3567_Atlan_Nov20/id01/20201014/ALIGN_2020_11_24_084524/spec/ALIGN_2020_11_24_084524.spec"
    ),
    "detector_data_path": (
        "/data/projects/carine/Experiments/ihhc3567_Atlan_Nov20/id01/20201014/ALIGN_2020_11_24_084524/detector/"
    ),
    "edf_file_template": "data_mpx4_%05d.edf.gz",
    "detector_name": "mpx4inr",

    # Optional paths
    "flat_field": None,
    "alien_mask": None,

    # REQUIRED: Directory to save all outputs
    #"dump_dir": os.getcwd() + f'/results/Pt/S183/',

    # # Preprocessing parameters
    # The Problem: The voxel_reference_methods you provided (["com", "max", "max"]) result in a final reference voxel located at (36, 347, 191).
    # The code then tries to crop a window of size preprocess_shape = (90, 150, 150) centered on this point.
    # The Calculation Failure: Let's look at the first axis (the rocking curve).
    # The reference point is at index 36.
    # To create a centered window of size 90, the code needs to take 45 voxels on each side of the reference.
    # The starting index of the crop would be 36 - 45 = -9. Since an array cannot have a negative index, the program fails and raises the ValueError.
    # The Solution: You must specify a preprocess_shape that is small enough to fit.
    # The maximum possible size for the first dimension, centered at index 36, would be 2 * 36 = 72.
    #"preprocess_shape": (72, 144, 144),
    #"voxel_reference_methods": ["com", "max", "max"],
    # after first run
    "voxel_reference_methods": [(40, 386, 163)],
    "preprocess_shape": (80, 200, 200),
    # after second run
    "voxel_size": (11.35, 8.12, 7.54),
    "background_level": 2,
    "hot_pixel_filter": True,
    "light_loading": False,

    # Detector calibration parameters
    "det_calib_params": {
      "cch1": 417.07983599290066,
      "cch2": 29.598791889093572,
      "detrot": -1.2384615384615385,
      "distance": 0.8675438689835612,
      "pwidth1": 5.5e-05,
      "pwidth2": 5.5e-05,
      "tilt": 2.204308447695206,
      "tiltazimuth": 240.6674276261259
    },
    
    # Other required parameters that will be filled with defaults if not provided
    "energy": 12996,
    "hkl": [0, 0, 2]
}
"""
"""
simulation_parameters = {
    "beamline_setup": "ID01BLISS",
    "scan": 227,
    "sample_name": "Pt_sapphire_0001",
    "experiment_file_path": "/data/projects/carine/Experiments/hc4050_high_res_Jan24/20240130/RAW_DATA/hc4050_id01.h5",
    "experiment_data_dir_path": None,
    'detector_data_path': None,
    'edf_file_template': None,
    "detector_name": "mpx1x4",
    "flatfield_path": None,
    "alien_mask": None,
    "dump_dir": "",
    "reconstruction_file": "mode.h5",
    "det_calib_params": {
        'cch1': 77.35933480238086,
        'cch2': 451.24824565419055,
        'pwidth1': 5.5e-05,
        'pwidth2': 5.5e-05,
        'distance': 0.994841865514257,
        'tiltazimuth': 46.7691571191134,
        'tilt': 4.585534011309222,
        'detrot': 0.28205128205128194,
    'outerangle_offset': 0.0
    },
    "hkl": [0, 0, 2],
}
"""
simulation_parameters = {
    "beamline_setup": "ID01BLISS",
    "scan": 227,
    "sample_name": "Pt_sapphire_0001",
    "experiment_file_path": "/data/projects/carine/Experiments/hc4050_high_res_Jan24/20240130/RAW_DATA/hc4050_id01.h5",
    "experiment_data_dir_path": None,
    'detector_data_path': None,
    'edf_file_template': None,
    "detector_name": "mpx1x4",
    "flatfield_path": None,
    "alien_mask": None,
    "dump_dir": "",
    "reconstruction_file": "mode.h5",
    "det_calib_params": None,
    "hkl": [0, 0, 2],
}


In [ ]:
notebook_data["hkl"] = simulation_parameters["hkl"]
%store notebook_data

globals().update(notebook_data)

In [ ]:
from ewoksdask import execute_graph
if not local_run:
    from ewoksdask.schedulers import slurm_scheduler
    cluster = slurm_scheduler(account="staff-id01",queue="gpu", walltime="1:00:00", memory="256GB", cores=num_cores, job_extra_directives=["--gres=gpu:1"])

In [ ]:
# or use default_inputs
nodes = [
    {
        "id": "init_pipeline",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.InitBcdiPipeline",
    },
    {
        "id": "preprocess",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.BcdiPreprocess",
    },
    {
        "id": "phase_retrieval",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.PhaseRetrieval",
    },
    {
        "id": "phase_analysis",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.PhaseAnalysis",
    },
    {
        "id": "select_best",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.SelectingBest",
    },
    {
        "id": "mode_decompose",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.ModeDecomposition",
    },
    {
        "id": "postprocess",
        "task_type": "class",
        "task_identifier": "facets_drivers.experiment.PostProcess",
    },
]

links = [
    {
        "source": "init_pipeline",
        "target": "preprocess",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
    {
        "source": "preprocess",
        "target": "phase_retrieval",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
    {
        "source": "phase_retrieval",
        "target": "phase_analysis",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
    {
        "source": "phase_analysis",
        "target": "select_best",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
    {
        "source": "select_best",
        "target": "mode_decompose",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
    {
        "source": "mode_decompose",
        "target": "postprocess",
        "data_mapping": [
            {"source_output": "pipeline", "target_input": "pipeline"},
            {"source_output": "scratch_directory", "target_input": "scratch_directory"}
        ],
    },
]

workflow = {"graph": {"id": "experiment"}, "nodes": nodes, "links": links}

inputs = [
    {"id": "init_pipeline", "name": "parameters", "value": simulation_parameters},
    {"id": "init_pipeline", "name": "scratch_directory", "value": experiment_directory},
]

In [ ]:
%%time
result = execute_graph(workflow, inputs=inputs, scheduler=None if local_run else cluster.scheduler_address)

In [ ]:
from pathlib import Path
ln_link = Path(f"/data/id01/inhouse/{user}/scratch/ewoks_01_experiment/structural_properties.vti")
ln_dest  = Path(f"/data/id01/inhouse/{user}/scratch/ewoks_01_experiment/S{simulation_parameters["scan"]}_structural_properties.vti")
ln_link.unlink(missing_ok=True)
ln_link.symlink_to(ln_dest)

In [ ]:
from IPython.display import display, Image

In [ ]:
[display(Image(data=figure)) for figure in result["figures"]]

In [ ]:
if not local_run:
    cluster.close()

## Simulating restart

In [ ]:
restarted_workflow = {
    "graph": {"id": "restarted_workflow"},
    "nodes": workflow["nodes"][5:],
    "links": workflow["links"][5:]
}

#restart = facets_drivers.experiment.pickle_to_rick(os.path.join(experiment_directory, "pipeline.pkl"))
restarted_inputs = [
    {"id": "mode_decompose", "name": "pipeline_path", "value": os.path.join(experiment_directory, "pipeline.pkl")},
    {"id": "mode_decompose", "name": "scratch_directory", "value": experiment_directory},
]